exploring various geemap capabilities

In [84]:
import ee
from geemap import geemap

In [85]:
Map = geemap.Map(center=(52, 28), zoom=8)
#Map.add_basemap('OpenStreetMap.BlackAndWhite')
#Map.add_basemap('SATELLITE')
Map

Map(center=[52, 28], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

get training data

In [86]:
fp_train_ext1 = "/home/markdj/Dropbox/artio/polesia/val/Vegetation_extent_rough.shp"
fp_roi = "/home/markdj/Dropbox/artio/polesia/val/roi_rough.shp"
fp_train = "/home/markdj/Dropbox/artio/polesia/val/Polesie-vegetation/Vegetation.shp"

In [87]:
roi = geemap.shp_to_ee(fp_roi)
train_ext = geemap.shp_to_ee(fp_train_ext1)
#train = geemap.shp_to_ee(fp_train)

visualisation is incredibly slow for the full training poly, probably dont do this.

In [ ]:
#poly.first().getInfo()
#Map.addLayer(train, {}, 'Vegetation')

get inputs

### look at GSW dataset
https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_3_GlobalSurfaceWater#bands

In [6]:
gsw = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select(['occurrence','seasonality','recurrence', 'max_extent'])

In [7]:
gsw_occ_vis = {
  'bands': ['occurrence'],
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', 'ffbbbb', '0000ff']
}
gsw_bin_vis = {
  'bands': ['max_extent'],
  'min': 0,
  'max': 1,
  'palette': ['ffffff', 'ffbbbb', '0000ff']
}
gsw_recurrence_vis = {
  'bands': ['recurrence'],
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', 'ffbbbb', '0000ff']
}
# Add Earth Engine DEM to map
Map.addLayer(gsw, gsw_occ_vis, 'occurence')
Map.addLayer(gsw, gsw_bin_vis, 'max_extent')
Map.addLayer(gsw, gsw_recurrence_vis, 'recurrence')

In [79]:
Map.addLayer(roi, {}, 'ROI')
Map.addLayer(train_ext, {}, 'train_ext')

### demo S2 imagery
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR


S2A cloudless product could be a good option: https://medium.com/google-earth/more-accurate-and-flexible-cloud-masking-for-sentinel-2-images-766897a9ba5f

In [88]:
def maskS2clouds(image):
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)


# rename bands
def renameS2bands(image):
    return image.rename('B', 'G', 'R', 'NIR', 'SWIR', 'SWIR2', 'QA60')


rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'], 
}

In [83]:
# Define filters for collection generation
filters = [#ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
           #ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
           #ee.Filter.equals("instrumentMode", "IW"),
           ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50), 
           ee.Filter.geometry(train_ext.geometry()),
           ee.Filter.date('2019-06-01', ee.Date('2019-06-01').advance(1, 'month'))
           ]

# var filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
#   .filter(ee.Filter.date('2019-01-01', '2020-01-01'))
#   .filter(ee.Filter.bounds(geometry))

# Load S2 collection with filters
s2 = ee.ImageCollection("COPERNICUS/S2_SR").filter(filters).map(maskS2clouds)
median = s2.median()
clipped = median.clip(train_ext.geometry())
Map.addLayer(clipped, rgbVis, 'Clipped')


In [89]:

# load Sentinel-2 TOA reflectance data.
s2collection = ee.ImageCollection('COPERNICUS/S2_SR') \
              .filterDate('2019-05-01', '2019-08-31') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
              .filterBounds(roi) \
              .select('B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60') \
              .map(maskS2clouds) #\
              #.map(renameS2bands)

#Reduce the first collection (median reducer in this case)
s2_median = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').median()
s2_min = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').min()
s2_max = s2collection.select('B2', 'B3', 'B4', 'B8', 'B11', 'B12').max()
s2_multiple = s2_median.addBands(s2_min).addBands(s2_max)

Map.addLayer(s2_median, rgbVis, 's2med')
Map.addLayer(s2_min, rgbVis, 's2min')
Map.addLayer(s2_max, rgbVis, 's2max')


## demo below here ##

In [70]:
# either pull the git repo, 
#git clone https://earthengine.googlesource.com/users/<user>/<repo>   (cant figure out pw used for this? not email?)
#or just copy paste from https://earthengine.googlesource.com/users/<mdejong>/
# convert a .js to a .py
from geemap.conversion import *

in_dir = '/home/markdj/repos/gee/test/'
js_to_python_dir(in_dir=in_dir, out_dir=in_dir, use_qgis=True)
print("Python scripts saved at: {}".format(in_dir))

Converting Earth Engine JavaScripts to Python scripts...

Processing 1/1: /home/markdj/repos/gee/test/prac3a.js
Python scripts saved at: /home/markdj/repos/gee/test/


In [23]:
m = geemap.Map()
m.basemap_demo()
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Eninge layers to Map
Map.addLayer(dem, vis_params, 'SRTM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(landsat7, {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0}, 'Landsat 7')
Map.addLayer(states, {}, "US States")

Map